Perintah `!pip install scikit-surprise` digunakan untuk menginstal pustaka Python bernama "scikit-surprise", yang dirancang untuk membangun dan menganalisis sistem rekomendasi.

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357275 sha256=6415914eb96fd9eeb11207b04751e14e04bf93868f48d1d790a05717a357ab83
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


Kode berikut mengimpor pustaka untuk manipulasi data, transformasi teks, pengukuran kesamaan, dan pembangunan sistem rekomendasi.

In [2]:
# Importing necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

Kode berikut memuat dataset contoh tentang buku dari URL (GoodReads) yang diberikan dan menyimpannya dalam variabel `data` menggunakan pustaka pandas.

In [3]:
# Project Overview
# Loading a sample dataset (Movies dataset)
url = 'https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv'
data = pd.read_csv(url)

Kode berikut mencetak jumlah total baris dan kolom dalam dataset serta daftar nama kolom yang ada.

In [4]:
# Data Understanding
print(f"Total Rows: {data.shape[0]}, Total Columns: {data.shape[1]}")
print(f"Dataset Columns: {data.columns.tolist()}")

Total Rows: 10000, Total Columns: 23
Dataset Columns: ['book_id', 'goodreads_book_id', 'best_book_id', 'work_id', 'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year', 'original_title', 'title', 'language_code', 'average_rating', 'ratings_count', 'work_ratings_count', 'work_text_reviews_count', 'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url', 'small_image_url']


Perintah `data.head()` menampilkan lima baris pertama dari dataset, memberikan gambaran awal tentang struktur dan konten data.

In [5]:
data.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


Perintah `print(data.isna().sum())` menghitung dan menampilkan jumlah nilai kosong (NaN) untuk setiap kolom dalam dataset, membantu mengidentifikasi masalah data yang perlu ditangani.

In [6]:
print(data.isna().sum())

book_id                         0
goodreads_book_id               0
best_book_id                    0
work_id                         0
books_count                     0
isbn                          700
isbn13                        585
authors                         0
original_publication_year      21
original_title                585
title                           0
language_code                1084
average_rating                  0
ratings_count                   0
work_ratings_count              0
work_text_reviews_count         0
ratings_1                       0
ratings_2                       0
ratings_3                       0
ratings_4                       0
ratings_5                       0
image_url                       0
small_image_url                 0
dtype: int64


Perintah berikut menggantikan nilai kosong di kolom `original_title` dengan nilai dari kolom `title`, memastikan tidak ada nilai hilang dalam kolom `original_title`.

In [7]:
data['original_title'] = data['original_title'].fillna(data['title'])

Kode berikut membuat kolom baru bernama `tags` yang menggabungkan informasi dari kolom `authors`, `original_title`, dan `average_rating` (dikonversi ke string) untuk digunakan dalam pemfilteran berbasis konten.

In [8]:
# Data Preparation
# Selecting relevant columns for the content-based filtering
data['tags'] = data['authors'] + " " + data['original_title'] + " " + data['average_rating'].astype(str)

Menampilkan atau melakukan pengecekan hasil dari kode sebelumnya.

In [9]:
data['tags'].head()

,tags
0,Suzanne Collins The Hunger Games 4.34
1,"J.K. Rowling, Mary GrandPré Harry Potter and t..."
2,Stephenie Meyer Twilight 3.57
3,Harper Lee To Kill a Mockingbird 4.25
4,F. Scott Fitzgerald The Great Gatsby 3.89


Memastikan tidak ada null value, atau missing value pada kolom tags.

In [10]:
print(data['tags'].isna().sum())

0


Kode berikut menggunakan `TfidfVectorizer` untuk mengubah kolom `tags` menjadi matriks TF-IDF, yang merepresentasikan teks secara numerik dan mengabaikan kata-kata umum (stop words) dalam bahasa Inggris.

In [11]:
# Content-Based Filtering
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['tags'])

Kode berikut menghitung matriks kesamaan kosinus antara semua item dalam matriks TF-IDF, yang digunakan untuk mengukur seberapa mirip satu item dengan item lainnya berdasarkan konten.

In [12]:
# Cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

Fungsi `content_based_recommendations` merekomendasikan buku berdasarkan kesamaan konten dengan cara mencari indeks buku yang diberikan, menghitung skor kesamaan, mengurutkannya, dan mengembalikan sepuluh judul buku teratas yang paling mirip.

In [13]:
# Function to recommend books based on content
def content_based_recommendations(title, cosine_sim=cosine_sim):
    idx = data.index[data['original_title'] == title].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    book_indices = [i[0] for i in sim_scores]
    return data['original_title'].iloc[book_indices]

Kode tersebut mencetak rekomendasi buku berdasarkan konten untuk judul "Harry Potter and the Order of the Phoenix" dengan menggunakan fungsi `content_based_recommendations`.

In [14]:
# Example of content-based recommendation
print("Content-Based Recommendations for 'Harry Potter':")
print(content_based_recommendations('Harry Potter and the Order of the Phoenix'))

Content-Based Recommendations for 'Harry Potter':
6140    Harry Potter and the Order of the Phoenix (Har...
3274    Harry Potter Boxed Set, Books 1-5 (Harry Potte...
23                    Harry Potter and the Goblet of Fire
22                Harry Potter and the Chamber of Secrets
1                Harry Potter and the Philosopher's Stone
26                 Harry Potter and the Half-Blood Prince
2100                     Harry Potter Boxed Set Books 1-4
24                   Harry Potter and the Deathly Hallows
3752         Harry Potter Collection (Harry Potter, #1-6)
17               Harry Potter and the Prisoner of Azkaban
Name: original_title, dtype: object


Kode untuk menghitung presisi melalui variable yang berisi semua buku Harry Potter yang ada pada dataset.

In [25]:
def calculate_precision(recommendations, actual_books_read):
  true_positives = len(set(recommendations) & set(actual_books_read))
  precision = true_positives / len(recommendations) if len(recommendations) > 0 else 0
  return precision

# Example usage:
recommended_books = content_based_recommendations('Harry Potter and the Order of the Phoenix').tolist()
# Let's assume the user actually read these books (replace with actual user data)
actual_books_read = ['Harry Potter and the Order of the Phoenix (Harry Potter, #5, Part 1)', 'Harry Potter Boxed Set, Books 1-5 (Harry Potter, #1-5)', 'Harry Potter and the Goblet of Fire', 'Harry Potter and the Chamber of Secrets', "Harry Potter and the Philosopher's Stone", 'Harry Potter and the Half-Blood Prince', 'Harry Potter Boxed Set Books 1-4', 'Harry Potter and the Deathly Hallows', 'Harry Potter Collection (Harry Potter, #1-6)', 'Harry Potter and the Prisoner of Azkaban']

precision = calculate_precision(recommended_books, actual_books_read)
print(f"Precision: {precision}")


Precision: 1.0


Kode ini memiliki fungsi untuk menghitung presisi dan recall dari model, dengan membandingkan buku buku yang ada pada user 2 (dimana user 2 ini telah membaca buku yang banyak pada dataset tersebut) ke buku yang direkomendasikan oleh model.

In [34]:
# Mengambil buku-buku relevan berdasarkan rating
def get_relevant_books(user_id, threshold=4):
    relevant_books = ratings[(ratings['user_id'] == user_id) & (ratings['rating'] >= threshold)]['book_id'].tolist()
    return relevant_books

# Menghitung Precision dari rekomendasi berbasis konten
def calculate_precision(user_id, recommended_books):
    relevant_books = get_relevant_books(user_id)
    relevant_and_recommended = [book for book in recommended_books if book in relevant_books]
    precision = len(relevant_and_recommended) / len(recommended_books) if recommended_books else 0
    return precision

# Menghitung Recall dari rekomendasi berbasis konten
def calculate_recall(user_id, recommended_books):
    relevant_books = get_relevant_books(user_id)
    relevant_and_recommended = [book for book in recommended_books if book in relevant_books]
    recall = len(relevant_and_recommended) / len(relevant_books) if relevant_books else 0
    return recall

# Memperbarui fungsi rekomendasi berbasis konten untuk menghitung precision dan recall
def content_based_recommendations_with_precision_recall(title, user_id, cosine_sim=cosine_sim):
    # Mendapatkan rekomendasi berbasis konten
    idx = data.index[data['original_title'] == title].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    book_indices = [i[0] for i in sim_scores]
    recommended_books = data['book_id'].iloc[book_indices].tolist()

    # Menghitung precision dan recall
    precision = calculate_precision(user_id, recommended_books)
    recall = calculate_recall(user_id, recommended_books)

    return data['original_title'].iloc[book_indices], precision, recall

# Contoh penggunaan fungsi dengan precision dan recall
title = 'Harry Potter and the Order of the Phoenix'
user_id = 2  # ID pengguna untuk evaluasi
recommended_books, precision, recall = content_based_recommendations_with_precision_recall(title, user_id)

print(f"Rekomendasi berbasis konten untuk '{title}':")
print(recommended_books)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")


Rekomendasi berbasis konten untuk 'Harry Potter and the Order of the Phoenix':
6140    Harry Potter and the Order of the Phoenix (Har...
3274    Harry Potter Boxed Set, Books 1-5 (Harry Potte...
23                    Harry Potter and the Goblet of Fire
22                Harry Potter and the Chamber of Secrets
1                Harry Potter and the Philosopher's Stone
26                 Harry Potter and the Half-Blood Prince
2100                     Harry Potter Boxed Set Books 1-4
24                   Harry Potter and the Deathly Hallows
3752         Harry Potter Collection (Harry Potter, #1-6)
17               Harry Potter and the Prisoner of Azkaban
Name: original_title, dtype: object
Precision: 0.70
Recall: 0.13


Kode berikut memuat dataset rating dari URL yang diberikan dan menyimpannya dalam variabel `ratings` menggunakan pustaka pandas.

In [16]:
# Collaborative Filtering
# Loading ratings dataset
url_ratings = 'https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/ratings.csv'
ratings = pd.read_csv(url_ratings)

Melihat 5 baris pertama pada dataframe ratings.

In [17]:
ratings.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


Kode berikut mempersiapkan dataset untuk pemfilteran kolaboratif dengan menggunakan `Reader` untuk mendefinisikan skala rating, dan memuat data dari DataFrame `ratings` ke dalam format yang dapat digunakan oleh pustaka Surprise.

In [18]:
# Preparing the dataset for collaborative filtering
reader = Reader(rating_scale=(1, 5))
data_collab = Dataset.load_from_df(ratings[['user_id', 'book_id', 'rating']], reader)

Kode berikut membagi dataset menjadi set pelatihan dan pengujian, dengan 80% data digunakan untuk pelatihan dan 20% untuk pengujian, menggunakan fungsi `train_test_split` dari pustaka Surprise.

In [19]:
# Train-test split
trainset, testset = train_test_split(data_collab, test_size=0.2)

Kode berikut menggunakan algoritma SVD (Singular Value Decomposition) untuk pelatihan model pemfilteran kolaboratif, kemudian menguji model tersebut pada set pengujian untuk menghasilkan prediksi rating.

In [20]:
# Using SVD for collaborative filtering
model = SVD()
model.fit(trainset)
predictions = model.test(testset)

Perintah `accuracy.rmse(predictions)` menghitung dan menampilkan root mean square error (RMSE) dari prediksi yang dihasilkan oleh model, yang digunakan untuk mengevaluasi akurasi model pemfilteran kolaboratif.

In [21]:
# Evaluation
accuracy.rmse(predictions)

RMSE: 0.8308


0.8308228044771095

Fungsi `collaborative_filtering_recommendations` merekomendasikan buku untuk pengguna tertentu dengan memprediksi rating untuk setiap buku, mengurutkan hasilnya, dan mengembalikan judul serta penulis dari buku-buku dengan rating tertinggi.

In [22]:
# Function to recommend books for a user based on collaborative filtering
def collaborative_filtering_recommendations(user_id, n_recommendations=10):
    book_ids = ratings['book_id'].unique()
    predicted_ratings = []

    for book_id in book_ids:
        predicted_ratings.append((book_id, model.predict(user_id, book_id).est))

    predicted_ratings.sort(key=lambda x: x[1], reverse=True)
    top_books = predicted_ratings[:n_recommendations]
    book_titles = data[data['book_id'].isin([book[0] for book in top_books])]

    return book_titles[['original_title', 'authors']]

Kode berikut mencetak rekomendasi buku berdasarkan pemfilteran kolaboratif untuk pengguna dengan ID 1, menggunakan fungsi `collaborative_filtering_recommendations`.

In [23]:
# Example of collaborative filtering recommendation
print("Collaborative Filtering Recommendations for User 1:")
print(collaborative_filtering_recommendations(1))

Collaborative Filtering Recommendations for User 1:
                                         original_title  \
463                      Holy Bible: King James Version   
722                                           Shantaram   
1098                                  道德經 [dào dé jīng]   
1347                                Horton Hears a Who!   
2080                                          Skin Game   
6589                The Authoritative Calvin and Hobbes   
6919  The Indispensable Calvin and Hobbes: A Calvin ...   
9140                           The Way of Kings, Part 1   
9181             Justice: What's the Right Thing to Do?   
9565  Attack of the Deranged Mutant Killer Monster S...   

                                                authors  
463                                           Anonymous  
722                               Gregory David Roberts  
1098  Lao Tzu, Gia-Fu Feng, Jane English, Chungliang...  
1347                                          Dr. Seuss  
2080    